In [1]:
# All imports
import carla
import time
import cv2
import numpy as np
import ultralytics
from ultralytics import YOLO

import sys
sys.path.append('C:\\Users\\eglm\\Documents\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

ModuleNotFoundError: No module named 'ultralytics'

In [12]:
!pip freeze


asttokens==2.4.1
backcall==0.2.0
carla==0.9.14
certifi==2023.7.22
charset-normalizer==3.3.1
colorama==0.4.6
comm==0.1.4
contourpy==1.1.1
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
executing==2.0.1
filelock==3.13.1
fonttools==4.43.1
fsspec==2023.10.0
idna==3.4
importlib-metadata==6.8.0
importlib-resources==6.1.0
ipykernel==6.26.0
ipython==8.12.3
jedi==0.19.1
Jinja2==3.1.2
jupyter_client==8.5.0
jupyter_core==5.5.0
kiwisolver==1.4.5
MarkupSafe==2.1.3
matplotlib==3.7.3
matplotlib-inline==0.1.6
mpmath==1.3.0
nest-asyncio==1.5.8
networkx==3.1
numpy==1.24.4
opencv-python==4.8.1.78
packaging==23.2
pandas==2.0.3
parso==0.8.3
pickleshare==0.7.5
Pillow==10.1.0
platformdirs==3.11.0
prompt-toolkit==3.0.39
psutil==5.9.6
pure-eval==0.2.2
py-cpuinfo==9.0.0
Pygments==2.16.1
pyparsing==3.1.1
python-dateutil==2.8.2
pytz==2023.3.post1
pywin32==306
PyYAML==6.0.1
pyzmq==25.1.1
requests==2.31.0
scipy==1.10.1
seaborn==0.13.0
six==1.16.0
stack-data==0.6.3
sympy==1.12
thop==0.1.1.post2209072238
torch==2.1.0

In [2]:
!pip install ultralytics

  Using cached matplotlib-3.7.3-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
  Using cached Pillow-10.1.0-cp38-cp38-win_amd64.whl.metadata (9.6 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
     ---------------------------------------- 0.5/42.2 MB 15.5 MB/s eta 0:00:03
     -- ------------------------------------- 2.9/42.2 MB 31.0 MB/s eta 0:00:02
     ---- ----------------------------------- 4.6/42.2 MB 32.5 MB/s eta 0:00:02
     -------- ------------------------------- 8.7/42.2 MB 46.3 MB/s eta 0:00:01
     ---------- ---------------------------- 11.7/42.2 MB 59.5 MB/s eta 0:00:01
     ----------- --------------------------- 12.6/42.2 MB 59.5 MB/s eta 0:00:01
     ------------- ------------------------- 14.4/42.2 MB 46.7 MB/s eta 0:00:01
     ------------------ -------------------- 19.8/42.2 MB 54.4 MB/s eta 0:00:01
     --------------------- ---------------- 24.1/42.2 MB 108.8 MB/s eta 0:00:

In [26]:
# Connect to the carla

client = carla.Client('localhost', 2000)

In [27]:
world = client.get_world()
spanw_points = world.get_map().get_spawn_points()

# look for a blueprint of mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

# spanw a car
start_point = spanw_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0],start_point)

# setup camera

CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans, attach_to= vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [28]:
point_a = start_point.location

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

# get de longest possible route
distance = 0
for loc in spanw_points:

    cur_route = grp.trace_route(point_a,loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color = carla.Color(r=0,g=255,b=0), life_time = 60.0,
        persistent_lines=True)

In [29]:
for way_point in route:
    vehicle.set_transform(way_point[0].transform)
    cv2.imshow('fake self-driving', camera_data['image'])
    cv2.waitKey(1)

time.sleep(2)
cv2.destroyAllWindows()
camera.stop()

# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()